In [ ]:
import json
import urllib.parse
import urllib.request
import requests

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Modeling
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Capstone-KPMG/Modeling
'Copy of Geo_edited.ipynb'   Data		  __pycache__
'Copy of Geo.ipynb'	     map_request_api.py   trafic_ratio.ipynb


In [ ]:
from map_request_api import *

#Start location is Exit 175 (the northest exit of Seatle) of I-5; end location is I-5 Exit 158 (the southest exit of Seatle)

In [ ]:
exits = pd.read_csv('Data/I5Exits.csv')
exits

,attributes.ExitNumber,geometry.x,geometry.y
0,1:00 AM,-122.672895,45.621707
1,1 C,-122.667187,45.627694
2,2,-122.661702,45.644626
3,3,-122.663286,45.654278
4,4,-122.664597,45.671567
...,...,...,...
723,NaN,-122.429282,47.234529
724,NaN,-122.432021,47.236890
725,NaN,-122.205107,47.470999
726,NaN,-122.454182,47.230811


In [ ]:
# End 158, Start 175
exits_list_1 = [str(i) for i in range(158,176)]
exits_list_2 = [str(i)+ " A" for i in range(158,176)]
exits_list_3 = [str(i)+ " B" for i in range(158,176)]
exits_list_3 = [str(i)+ " AB" for i in range(158,176)]
exits = exits[exits['attributes.ExitNumber'].str.split(expand=True).iloc[:,0].isin(exits_list_1)]
exits = exits.drop_duplicates(subset='attributes.ExitNumber', keep="last")
exits.columns = ['attributes.ExitNumber','geometry.y','geometry.x']
exits

,attributes.ExitNumber,geometry.y,geometry.x
74,163,-122.320626,47.566044
75,164 B,-122.320756,47.589947
76,165,-122.326825,47.603753
82,169,-122.322342,47.656009
83,170,-122.321720,47.670035
219,161,-122.317089,47.551649
221,163 A,-122.319557,47.577785
222,163 B,-122.319302,47.579356
223,165 A,-122.331153,47.611457
224,165 B,-122.330623,47.612731


# For each exit, find number of EV stations and sum of level 2 and fast charger numbers

In [ ]:
def getRemoteEvDataList(df,lt,lg,name):

    df['shapePoints'] = df.apply(lambda x: [x[lt],x[lg]],axis=1)
    remoteDataList = []
    for row in range(df.shape[0]):
        inner = {}
        r_data = df.iloc[row]
        inner['name'] = str(int(r_data['EV_Level2_EVSE_Num'] + r_data['EV_DC_Fast_Count']))
        inner['key'] = str(row)
        inner['shapePoints'] = r_data['shapePoints']
        remoteDataList.append(inner)
    return remoteDataList

# fill missing number of chargers to 0

In [ ]:
ev_station = pd.read_csv('Data/alt_fuel_stations(Sep_25_2022).csv')
ev_station[['EV_Level2_EVSE_Num', 'EV_DC_Fast_Count']] = ev_station[['EV_Level2_EVSE_Num','EV_DC_Fast_Count']].fillna(value=0)
remoteDataListEv = getRemoteEvDataList(ev_station,'Latitude','Longitude','Station_Name')

In [ ]:
def get_avg_from_result(result):
    filtered = pd.DataFrame(result['searchResults'])
    #print(filtered['name'].astype(int).mean())
    return filtered['key'], filtered['shapePoints'],  filtered['name'].astype(int).mean()

def get_max_from_result(result):
    filtered = pd.DataFrame(result['searchResults'])
    #print(filtered['name'].astype(int).mean())
    return filtered['key'], filtered['shapePoints'],  filtered['name'].astype(int).max()

def get_sum_from_result(result):
    filtered = pd.DataFrame(result['searchResults'])
    #print(filtered['name'].astype(int).mean())
    return filtered['key'], filtered['shapePoints'],  filtered['name'].astype(int).sum()

In [ ]:
# for each exit, compute its total EV Stations and total chargers
exit_ev_data = []
for exit_i in range(exits.shape[0]):
  exit_data = exits.iloc[exit_i]
  exits_origin = [exit_data['geometry.x'], exit_data['geometry.y']]

  # 1 mile radius ev station and ev charger count
  result_r1 = post_radius_search_request(exits_origin, remoteDataListEv, 1)

  if result_r1['resultsCount'] != 0:
    ev_station_r1_count = result_r1['resultsCount']
    ev_name_1, ev_shape_1, ev_chargers_total_cnt_1 = get_sum_from_result(result_r1)

  else:
    ev_station_r1_count = 0
    ev_chargers_total_cnt_1 = 0

  # 2 mile radius ev station and ev charger count
  result_r2 = post_radius_search_request(exits_origin, remoteDataListEv, 2)
  if result_r2['resultsCount'] != 0:
    ev_station_r2_count = result_r2['resultsCount']
    ev_name_2, ev_shape_2, ev_chargers_total_cnt_2 = get_sum_from_result(result_r2)

  else:
    ev_station_r2_count = 0
    ev_chargers_total_cnt_2 = 0

  # 5 mile radius ev station and ev charger count
  result_r5 = post_radius_search_request(exits_origin, remoteDataListEv, 5)
  if result_r5['resultsCount'] != 0:
    ev_station_r5_count = result_r5['resultsCount']
    ev_name_5, ev_shape_5, ev_chargers_total_cnt_5 = get_sum_from_result(result_r5)
  else:
    ev_station_r5_count = 0
    ev_chargers_total_cnt_5 = 0

  # print(result_r5)

  
  exit_ev_data.append([exit_data['attributes.ExitNumber'], exit_data['geometry.x'], exit_data['geometry.y'], 
                       ev_station_r1_count, ev_chargers_total_cnt_1, ev_station_r2_count, ev_chargers_total_cnt_2, ev_station_r5_count, ev_chargers_total_cnt_5])

In [ ]:
exit_ev_data_df = pd.DataFrame(exit_ev_data)
exit_ev_data_df.columns = ['I5 Exit Name','Exit Lat','Exit Long','num_ev_station_cnt_1mile','num_ev_charger_cnt_1mile','num_ev_station_cnt_2mile','num_ev_charger_cnt_2mile','num_ev_station_cnt_5mile','num_ev_charger_cnt_5mile']
exit_ev_data_df

,I5 Exit Name,Exit Lat,Exit Long,num_ev_station_cnt_1mile,num_ev_charger_cnt_1mile,num_ev_station_cnt_2mile,num_ev_charger_cnt_2mile,num_ev_station_cnt_5mile,num_ev_charger_cnt_5mile
0,163,47.566044,-122.320626,1,5,16,31,254,633
1,164 B,47.589947,-122.320756,27,76,97,288,262,655
2,165,47.603753,-122.326825,85,262,219,558,277,694
3,169,47.656009,-122.322342,13,27,34,71,285,732
4,170,47.670035,-122.321720,12,21,28,57,275,708
5,161,47.551649,-122.317089,4,9,14,33,196,499
6,163 A,47.577785,-122.319557,4,11,37,106,251,629
7,163 B,47.579356,-122.319302,8,15,46,123,253,636
8,165 A,47.611457,-122.331153,169,455,225,573,275,695
9,165 B,47.612731,-122.330623,164,418,224,571,275,695


# For each exit, find max number of traffic

In [ ]:
trafficCount = pd.read_csv('Data/WSDOT_-_Traffic_Counts_(AADT).csv')
trafficCount.head(5)

,X,Y,OBJECTID,LRSDate,RouteIdentifier,StateRouteNumber,RelatedRouteType,RelatedRouteQualifier,AccumulatedRouteMile,AADT,SingleUnitTruckPct,DoubleUnitTruckPct,TripleUnitTruckPct,DirectionOfTravel,Location,GlobalID,ReportingYear
0,-122.365641,48.240274,1,2021/12/31 00:00:00+00,532,532,NaN,NaN,4.26,19000,NaN,NaN,NaN,Bothways,After Milepost 4.25 A: RIGHT INTERSECTION 98TH...,{9B3865D4-8261-4196-A946-13A1955B8AF5},2021
1,-122.253938,47.298320,2,2021/12/31 00:00:00+00,167Q101469,167,Q1,01469,0.20,3700,NaN,NaN,NaN,Northbound,On Q1 RAMP (15TH ST SW TO SR 167 NB) After RIG...,{0D9D296A-1DFD-4439-9EEA-B3584A94167F},2021
2,-118.387595,46.027572,3,2021/12/31 00:00:00+00,125,125,NaN,NaN,1.89,16000,NaN,NaN,NaN,Bothways,After Milepost 1.88 A: LEFT INTERSECTION COLLE...,{5A643190-D5CC-4A33-8322-661CEBA58B7F},2021
3,-122.541535,46.937836,4,2021/12/31 00:00:00+00,702,702,NaN,NaN,0.59,8700,4.44,2.13,0.31,Bothways,After Milepost 0.58 A: RIGHT INTERSECTION HART...,{1F772D26-2ECC-4503-9A93-365F86B75653},2021
4,-122.333841,47.545449,5,2021/12/31 00:00:00+00,099S102690,99,S1,02690,0.13,15000,NaN,NaN,NaN,Southbound,On S1 RAMP (E MARGNL WAY TO SR 99 SB) After RI...,{0E0F487B-F425-4C64-BF26-5ECA88B40456},2021


In [ ]:
def getRemoteTrafDataList(df,lt,lg,name):

    df['shapePoints'] = df.apply(lambda x: [x[lt],x[lg]],axis=1)
    remoteDataList = []
    for row in range(df.shape[0]):
        inner = {}
        r_data = df.iloc[row]
        inner['name'] = str(r_data['AADT'])
        inner['key'] = str(row)
        inner['shapePoints'] = r_data['shapePoints']
        remoteDataList.append(inner)
    return remoteDataList

In [ ]:
remoteDataListTraffic =  getRemoteTrafDataList(trafficCount, 'Y', 'X', 'AADT')

In [ ]:
# for each exit, compute its max trafic
exit_trafic_data = []
for exit_i in range(exits.shape[0]):
  exit_data = exits.iloc[exit_i]
  exits_origin = [str(exit_data['geometry.x']), str(exit_data['geometry.y'])]

  # 1 mile radius ev station and ev charger count
  result_r1 = post_radius_search_request(exits_origin, remoteDataListTraffic, 1)

  if result_r1['resultsCount'] != 0:
    traffic_r1_count = result_r1['resultsCount']
    key_1m_max, shape_1m_max, traff_cnt_1m_max = get_max_from_result(result_r1)

  else:
    traffic_r1_count = 0
    traff_cnt_1m_max = 0

  # 2 mile radius ev station and ev charger count
  result_r2 = post_radius_search_request(exits_origin, remoteDataListTraffic, 2)
  if result_r2['resultsCount'] != 0:
    traffic_r2_count = result_r2['resultsCount']
    key_2m_max, shape_2m_max, traff_cnt_2m_max = get_max_from_result(result_r2)

  else:
    traffic_r2_count = 0
    traff_cnt_2m_max = 0

  # 5 mile radius ev station and ev charger count
  result_r5 = post_radius_search_request(exits_origin, remoteDataListTraffic, 5)
  if result_r5['resultsCount'] != 0:
    traffic_r5_count = result_r5['resultsCount']
    key_5m_max, shape_5m_max, traff_cnt_5m_max = get_max_from_result(result_r5)

  else:
    traffic_r5_count = 0
    traff_cnt_5m_max = 0

  # print(result_r5)

  
  exit_trafic_data.append([exit_data['attributes.ExitNumber'], 
                       traff_cnt_1m_max, traff_cnt_2m_max, traff_cnt_5m_max])

In [ ]:
exit_trafic_data_df = pd.DataFrame(exit_trafic_data)
exit_trafic_data_df.columns = ['I5 Exit Name','max_traffic_count_1mile','avg_traffic_count_1mile','max_traffic_count_2mile','avg_traffic_count_1mile','max_traffic_count_5mile','avg_traffic_count_1mile']
exit_trafic_data_df

,I5 Exit Name,max_traffic_count_1mile,max_traffic_count_2mile,max_traffic_count_5mile
0,163,240000,240000,240000
1,164 B,240000,240000,240000
2,165,190000,240000,240000
3,169,189000,201000,240000
4,170,173000,189000,201000
5,161,234000,240000,240000
6,163 A,240000,240000,240000
7,163 B,240000,240000,240000
8,165 A,190000,240000,240000
9,165 B,190000,201000,240000


In [ ]:
main_df = exit_ev_data_df.merge(exit_trafic_data_df, left_on='I5 Exit Name', right_on='I5 Exit Name')

# Calculate ratio

In [ ]:
main_df

,I5 Exit Name,Exit Lat,Exit Long,num_ev_station_cnt_1mile,num_ev_charger_cnt_1mile,num_ev_station_cnt_2mile,num_ev_charger_cnt_2mile,num_ev_station_cnt_5mile,num_ev_charger_cnt_5mile,max_traffic_count_1mile,max_traffic_count_2mile,max_traffic_count_5mile
0,163,47.566044,-122.320626,1,5,16,31,254,633,240000,240000,240000
1,164 B,47.589947,-122.320756,27,76,97,288,262,655,240000,240000,240000
2,165,47.603753,-122.326825,85,262,219,558,277,694,190000,240000,240000
3,169,47.656009,-122.322342,13,27,34,71,285,732,189000,201000,240000
4,170,47.670035,-122.321720,12,21,28,57,275,708,173000,189000,201000
5,161,47.551649,-122.317089,4,9,14,33,196,499,234000,240000,240000
6,163 A,47.577785,-122.319557,4,11,37,106,251,629,240000,240000,240000
7,163 B,47.579356,-122.319302,8,15,46,123,253,636,240000,240000,240000
8,165 A,47.611457,-122.331153,169,455,225,573,275,695,190000,240000,240000
9,165 B,47.612731,-122.330623,164,418,224,571,275,695,190000,201000,240000


In [ ]:
main_df['traffic/ev_station_1mile'] = main_df['max_traffic_count_1mile'].astype(int)/main_df['num_ev_station_cnt_1mile'].astype(int)

In [ ]:
main_df['traffic/ev_charger_1mile'] = main_df['max_traffic_count_1mile'].astype(int)/main_df['num_ev_charger_cnt_1mile'].astype(int)

In [ ]:
main_df['traffic/ev_station_2mile'] = main_df['max_traffic_count_2mile'].astype(int)/main_df['num_ev_station_cnt_2mile'].astype(int)

In [ ]:
main_df['traffic/ev_charger_2mile'] = main_df['max_traffic_count_2mile'].astype(int)/main_df['num_ev_charger_cnt_2mile'].astype(int)

In [ ]:
main_df['traffic/ev_station_5mile'] = main_df['max_traffic_count_5mile'].astype(int)/main_df['num_ev_station_cnt_5mile'].astype(int)

In [ ]:
main_df['traffic/ev_charger_5mile'] = main_df['max_traffic_count_5mile'].astype(int)/main_df['num_ev_charger_cnt_5mile'].astype(int)

In [ ]:
main_df

,I5 Exit Name,Exit Lat,Exit Long,num_ev_station_cnt_1mile,num_ev_charger_cnt_1mile,num_ev_station_cnt_2mile,num_ev_charger_cnt_2mile,num_ev_station_cnt_5mile,num_ev_charger_cnt_5mile,max_traffic_count_1mile,max_traffic_count_2mile,max_traffic_count_5mile,traffic/ev_station_1mile,traffic/ev_charger_1mile,traffic/ev_station_2mile,traffic/ev_charger_2mile,traffic/ev_station_5mile,traffic/ev_charger_5mile
0,163,47.566044,-122.320626,1,5,16,31,254,633,240000,240000,240000,2.400000e+05,4.800000e+04,15000.000000,7741.935484,944.881890,379.146919
1,164 B,47.589947,-122.320756,27,76,97,288,262,655,240000,240000,240000,8.888889e+03,3.157895e+03,2474.226804,833.333333,916.030534,366.412214
2,165,47.603753,-122.326825,85,262,219,558,277,694,190000,240000,240000,2.235294e+03,7.251908e+02,1095.890411,430.107527,866.425993,345.821326
3,169,47.656009,-122.322342,13,27,34,71,285,732,189000,201000,240000,1.453846e+04,7.000000e+03,5911.764706,2830.985915,842.105263,327.868852
4,170,47.670035,-122.321720,12,21,28,57,275,708,173000,189000,201000,1.441667e+04,8.238095e+03,6750.000000,3315.789474,730.909091,283.898305
5,161,47.551649,-122.317089,4,9,14,33,196,499,234000,240000,240000,5.850000e+04,2.600000e+04,17142.857143,7272.727273,1224.489796,480.961924
6,163 A,47.577785,-122.319557,4,11,37,106,251,629,240000,240000,240000,6.000000e+04,2.181818e+04,6486.486486,2264.150943,956.175299,381.558029
7,163 B,47.579356,-122.319302,8,15,46,123,253,636,240000,240000,240000,3.000000e+04,1.600000e+04,5217.391304,1951.219512,948.616601,377.358491
8,165 A,47.611457,-122.331153,169,455,225,573,275,695,190000,240000,240000,1.124260e+03,4.175824e+02,1066.666667,418.848168,872.727273,345.323741
9,165 B,47.612731,-122.330623,164,418,224,571,275,695,190000,201000,240000,1.158537e+03,4.545455e+02,897.321429,352.014011,872.727273,345.323741


In [ ]:
# main_df.drop(16, inplace=True)
# main_df.drop(19, inplace=True)

In [ ]:
main_df

,I5 Exit Name,Exit Lat,Exit Long,num_ev_station_cnt_1mile,num_ev_charger_cnt_1mile,num_ev_station_cnt_2mile,num_ev_charger_cnt_2mile,num_ev_station_cnt_5mile,num_ev_charger_cnt_5mile,max_traffic_count_1mile,max_traffic_count_2mile,max_traffic_count_5mile,traffic/ev_station_1mile,traffic/ev_charger_1mile,traffic/ev_station_2mile,traffic/ev_charger_2mile,traffic/ev_station_5mile,traffic/ev_charger_5mile
0,163,47.566044,-122.320626,1,5,16,31,254,633,240000,240000,240000,240000.000000,48000.000000,15000.000000,7741.935484,944.881890,379.146919
1,164 B,47.589947,-122.320756,27,76,97,288,262,655,240000,240000,240000,8888.888889,3157.894737,2474.226804,833.333333,916.030534,366.412214
2,165,47.603753,-122.326825,85,262,219,558,277,694,190000,240000,240000,2235.294118,725.190840,1095.890411,430.107527,866.425993,345.821326
3,169,47.656009,-122.322342,13,27,34,71,285,732,189000,201000,240000,14538.461538,7000.000000,5911.764706,2830.985915,842.105263,327.868852
4,170,47.670035,-122.321720,12,21,28,57,275,708,173000,189000,201000,14416.666667,8238.095238,6750.000000,3315.789474,730.909091,283.898305
5,161,47.551649,-122.317089,4,9,14,33,196,499,234000,240000,240000,58500.000000,26000.000000,17142.857143,7272.727273,1224.489796,480.961924
6,163 A,47.577785,-122.319557,4,11,37,106,251,629,240000,240000,240000,60000.000000,21818.181818,6486.486486,2264.150943,956.175299,381.558029
7,163 B,47.579356,-122.319302,8,15,46,123,253,636,240000,240000,240000,30000.000000,16000.000000,5217.391304,1951.219512,948.616601,377.358491
8,165 A,47.611457,-122.331153,169,455,225,573,275,695,190000,240000,240000,1124.260355,417.582418,1066.666667,418.848168,872.727273,345.323741
9,165 B,47.612731,-122.330623,164,418,224,571,275,695,190000,201000,240000,1158.536585,454.545455,897.321429,352.014011,872.727273,345.323741


In [ ]:
# traffic/EV station in 1 mile
main_df['traffic/ev_station_1mile'].mean()*0.02

651.1960009738859

In [ ]:
# traffic/EV charger in 1 mile
main_df['traffic/ev_charger_1mile'].mean()*0.02

223.57428393576143

In [ ]:
# traffic/EV station in 2 mile
main_df['traffic/ev_station_2mile'].mean()*0.02

235.74523384769017

In [ ]:
# traffic/EV charger in 2 mile
main_df['traffic/ev_charger_2mile'].mean()*0.02 

63.31042720630594

In [ ]:
# traffic/EV station in 5 mile
main_df['traffic/ev_station_5mile'].mean()*0.02  

34.20518469406953

In [ ]:
# traffic/EV charger in 5 mile
main_df['traffic/ev_charger_5mile'].mean()*0.02  

11.509117647905283